<a href="https://colab.research.google.com/github/selgebali/Colabs/blob/main/RTGs_KGs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knowledge Graph Visualization using DataCite API

## Overview
This project is a Python-based tool that fetches metadata from the DataCite API and visualizes relationships among research resources in the form of an interactive knowledge graph. The tool creates directed graphs to map relationships between research objects (like studies, datasets, and publications), their creators, contributors, publishers, and related items. The visualization is implemented using `Plotly` for enhanced interactivity.

The generated graphs and data nodes/edges are saved to HTML and CSV files, respectively, allowing users to visualize connections and explore research data in a structured manner.

## Features
- **Data Extraction from DataCite API**: The tool retrieves metadata for a specified resource type, allowing researchers to understand relationships across resources.
- **Customizable Resource Types**: The type of resource to be fetched (e.g., datasets, publications) can be easily specified.
- **Interactive Graphs with Plotly**: Relationships between resources are visualized using `Plotly`, offering an intuitive way to understand the interconnected nature of the metadata.
- **Data Export**: Nodes and edges of the graph are saved to CSV files, enabling further analysis.
- **HTML Graph Merging**: Each generated graph is appended to a single HTML file, allowing users to view multiple visualizations in one document.

## Prerequisites
To use this project, you need the following:
- Python 3.x
- `requests` (for making HTTP requests to the DataCite API)
- `pandas` (for handling tabular data)
- `networkx` (for creating graph data structures)
- `plotly` (for graph visualization)

You can install all dependencies by running:

```sh
pip install requests pandas networkx plotly
```

## Usage

### 1. Fetch Metadata from the DataCite API
The script fetches data from the DataCite API, specifically targeting resources of a particular type (e.g., study-registration). The user can modify the resource type and the limit of the resources fetched by changing the following variables:

```python
resource_type = "study-registration"
limit = 20  # Set limit to None to fetch all available resources
```
The script will use the `fetch_api_data()` function to extract metadata and parse it to create a structured list of resources.

### 2. Generate Knowledge Graph Visualizations
Once the metadata is fetched, it is visualized using `visualize_resource_plotly()`. Each graph shows relationships among key components of the resource:
- **Central Node**: Represents the main resource, including details such as DOI and resource type.
- **Creators, Contributors, Publishers, Related Items**: Visualized as additional nodes connected to the central resource node.

### 3. Export Data to Files
- **HTML File**: All generated graphs are saved in an HTML file (`merged_knowledge_graphs.html`), which allows easy access and visualization of all resources.
- **CSV Files**: Nodes and edges are exported to CSV files (`merged_nodes.csv` and `merged_edges.csv`) for further data analysis.

### 4. Run the Script
The script can be run as follows:

```sh
python script_name.py
```
It initializes an HTML file, fetches resources from the DataCite API, and appends visualizations to the HTML file. Each resource and its relationships are visualized and stored.

### Example Usage
To modify the example usage, adjust the variables at the beginning of the script:

```python
resource_type = "dataset"  # Specify a different resource type
limit = 10  # Limit the number of resources to visualize
api_url = f"https://api.datacite.org/dois?resource-type-id={resource_type.lower()}&page[size]={limit if limit else 100}"
```

### Customization
- **Resource Type**: You can change the `resource_type` variable to fetch a different type of resource.
- **Graph Appearance**: The visualization colors and styles are customizable via the `custom_colors` array and the `visualize_resource_plotly()` function parameters.

## Output
- **HTML File (`merged_knowledge_graphs.html`)**: Contains interactive knowledge graphs for each resource.
- **Nodes CSV (`merged_nodes.csv`)**: Lists nodes present in the knowledge graphs, including metadata such as size and color.
- **Edges CSV (`merged_edges.csv`)**: Lists relationships between nodes (edges) in the graph, including relationship types.

## Debug Logs
The script includes detailed debug logs to trace the status of data fetching, parsing, and graph visualization processes. These logs help in troubleshooting and understanding each step of the execution.

## Contribution and Further Development
- **Adding More Metadata Fields**: You can enhance the script by adding more metadata fields to the nodes or implementing advanced analysis features.
- **Graph Styling**: More advanced layouts and styling features could be implemented using `plotly` or `networkx`.




In [ ]:
import requests
import json
import pandas as pd
import networkx as nx
import plotly.graph_objs as go
import plotly.io as pio

# Custom color palette
custom_colors = ['#243B54', '#00B1E2', '#5B88B9', '#46BCAB', '#90D7CD', '#BC2B66']

# Function to fetch data from the API endpoint with dynamic resource type
def fetch_api_data(url, resource_type, limit=None):
    resources = []
    while url:
        response = requests.get(url)  # Send a request to the given API URL
        print(f"Fetching data from URL: {url}")  # Debug log: API URL being fetched
        response.raise_for_status()  # Raise an error if the request was unsuccessful
        data = response.json()  # Parse the JSON response
        print(f"API response status code: {response.status_code}")  # Debug log: Status code of response
        resources.extend(parse_resources(data['data'], resource_type))  # Parse the resources
        url = data['links'].get('next') if limit is None else None  # Update URL to next page if no limit specified
        if limit and len(resources) >= limit:
            resources = resources[:limit]  # Limit the number of resources if a limit is specified
            break
    total_resources = data['meta'].get('total', 'Unknown')  # Get total number of resources from metadata
    print(f"Total number of resources available: {total_resources}")  # Print the total number of resources available
    print(f"Number of resources fetched: {len(resources)}")  # Debug log: Number of resources fetched
    return resources

# Function to parse resources
def parse_resources(data, resource_type):
    resources = []
    for item in data:
        # Normalize resourceTypeGeneral to handle variations like hyphens and casing
        resource_type_general = item['attributes']['types'].get('resourceTypeGeneral', '').replace('-', '').lower()
        if resource_type_general == resource_type.replace('-', '').lower():
            resource_data = {
                "doi": item['attributes'].get('doi', 'No DOI available'),
                "resourceTypeGeneral": item['attributes']['types'].get('resourceTypeGeneral', 'No resourceTypeGeneral available'),
                "resourceType": item['attributes']['types'].get('resourceType', 'No resource type available'),
                "schemaOrg": item['attributes']['types'].get('schemaOrg', 'No schemaOrg available'),
                "creators": [],
                "contributors": [],
                "publishers": item['attributes'].get('publisher', 'No publisher information'),
                "relatedItems": []
            }
            # Add creators to the resource data
            for creator in item['attributes'].get('creators', []):
                resource_data['creators'].append({
                    "name": creator.get('name', 'No name available'),
                    "identifier": creator['nameIdentifiers'][0]['nameIdentifier'] if (creator.get('nameIdentifiers') and creator['nameIdentifiers']) else 'No identifier'
                })
            # Add contributors to the resource data
            for contributor in item['attributes'].get('contributors', []):
                resource_data['contributors'].append({
                    "name": contributor.get('name', 'No name available'),
                    "type": contributor.get('contributorType', 'No type available'),
                    "identifier": contributor['nameIdentifiers'][0]['nameIdentifier'] if (contributor.get('nameIdentifiers') and contributor['nameIdentifiers']) else 'No identifier'
                })
            # Add related items if they exist
            if 'relatedIdentifiers' in item['attributes']:
                for related in item['attributes']['relatedIdentifiers']:
                    resource_data['relatedItems'].append({
                        "identifier": related.get('relatedIdentifier', 'No identifier available'),
                        "relationType": related.get('relationType', 'No relation type available')
                    })
            resources.append(resource_data)  # Append parsed resource data to the list
    print(f"Parsed {len(resources)} resources.")  # Debug log: Number of resources parsed
    return resources

# Function to visualize the resource network and append results
def visualize_resource_plotly(resource, html_file, nodes_csv, edges_csv):
    G = nx.DiGraph()  # Create a directed graph

    # Update central node to include additional information
    resource_node_label = f"{resource['resourceTypeGeneral']}:<br>DOI: {resource['doi']} <br> Resource Type: {resource['resourceType']} <br> SchemaOrg: {resource['schemaOrg']}"
    G.add_node(resource_node_label, label=resource_node_label, size=400, color=custom_colors[0])
    print(f"Added central node: {resource_node_label}")  # Debug log: Central node added

    # Add creators
    for creator in resource["creators"]:
        creator_label = f"Creator: {creator['name']} <br> ID: {creator['identifier']}"
        G.add_node(creator_label, label=creator_label, size=300, color=custom_colors[1])
        G.add_edge(resource_node_label, creator_label, relationType='Created by')
        print(f"Added creator node and edge: {creator_label}")  # Debug log: Creator node and edge added

    # Add contributors
    for contributor in resource["contributors"]:
        contributor_label = f"Contributor: {contributor['name']} <br> Type: {contributor['type']} <br> ID: {contributor['identifier']}"
        G.add_node(contributor_label, label=contributor_label, size=300, color=custom_colors[2])
        G.add_edge(resource_node_label, contributor_label, relationType='Contributed by')
        print(f"Added contributor node and edge: {contributor_label}")  # Debug log: Contributor node and edge added

    # Add publisher
    publisher_label = f"Publisher: <br> {resource['publishers']}"
    G.add_node(publisher_label, label=publisher_label, size=300, color=custom_colors[3])
    G.add_edge(resource_node_label, publisher_label, relationType='Published by')
    print(f"Added publisher node and edge: {publisher_label}")  # Debug log: Publisher node and edge added

    # Add related items
    for item in resource["relatedItems"]:
        item_node_label = f"{item['relationType']}: <br> {item['identifier']}"
        G.add_node(item_node_label, label=item_node_label, size=300, color=custom_colors[4])
        G.add_edge(resource_node_label, item_node_label, relationType=item["relationType"])
        print(f"Added related item node and edge: {item_node_label}")  # Debug log: Related item node and edge added

    # Generate positions for the nodes
    pos = nx.spring_layout(G, seed=42)

    node_x, node_y, node_text, node_hovertext, node_sizes, node_colors = [], [], [], [], [], []
    for node in G.nodes(data=True):
        x, y = pos[node[0]]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node[1]['label'])
        node_hovertext.append(node[1]['label'])
        node_sizes.append(node[1]['size'])
        node_colors.append(node[1]['color'])

    # Prepare edge attributes
    edge_x, edge_y, edge_text = [], [], []
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    # Create edge trace
    edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=3, color='gray'), hoverinfo='none', mode='lines')

    # Create node trace
    node_trace = go.Scatter(
        x=node_x, y=node_y, text=node_text, hovertext=node_hovertext, mode='markers+text',
        textposition='middle center', marker=dict(size=node_sizes, color=node_colors, opacity=1.0),
            textfont=dict(family="Arial", size=18, color="white", weight='bold')
    )

    # Create the Plotly figure
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title=f"Knowledge Graph of {resource['doi']}",
                        titlefont_size=16, showlegend=False, hovermode='closest', margin=dict(b=20, l=5, r=5, t=40),
                        width=1500, height=1000, xaxis=dict(showgrid=False, zeroline=False, showticklabels=False), yaxis=dict(showgrid=False,  zeroline=False, showticklabels=False),
                        plot_bgcolor='white',  # Set plot background color here
                        paper_bgcolor='white'  # Set overall figure background color
                    )
                      )
    fig.show()
    print(f"Displayed knowledge graph for resource: {resource['doi']}")  # Debug log: Graph displayed

    # Append to HTML file
    with open(html_file, 'a') as f:
        f.write(pio.to_html(fig, include_plotlyjs='cdn'))
        print(f"Appended graph to HTML file: {html_file}")  # Debug log: Graph appended to HTML

    # Append nodes to CSV
    node_data = [{"Node": node[0], "Label": node[1]['label'], "Size": node[1]['size'], "Color": node[1]['color']} for node in G.nodes(data=True)]
    nodes_df = pd.DataFrame(node_data)
    nodes_df.to_csv(nodes_csv, mode='a', index=False, header=not nodes_df.empty, encoding='utf-8')
    print(f"Appended nodes to CSV file: {nodes_csv}")  # Debug log: Nodes appended to CSV

    # Append edges to CSV
    edge_data = [{"Source": edge[0], "Target": edge[1], "RelationType": edge[2].get('relationType', 'Related to')} for edge in G.edges(data=True)]
    edges_df = pd.DataFrame(edge_data)
    edges_df.to_csv(edges_csv, mode='a', index=False, header=not edges_df.empty, encoding='utf-8')
    print(f"Appended edges to CSV file: {edges_csv}")  # Debug log: Edges appended to CSV

# Example usage
resource_type = "study-registration"
limit = 20  # Set limit to None to fetch all available resources
api_url = f"https://api.datacite.org/dois?resource-type-id={resource_type.lower()}&page[size]={limit if limit else 100}"

html_file = "merged_knowledge_graphs.html"
nodes_csv = "merged_nodes.csv"
edges_csv = "merged_edges.csv"

# Initialize HTML file with basic structure
with open(html_file, 'w') as f:
    f.write("<html><head><title>Knowledge Graphs</title></head><body>")
    print(f"Initialized HTML file: {html_file}")  # Debug log: HTML file initialized

resources = fetch_api_data(api_url, resource_type, limit=limit)
print(f"Number of resources to visualize: {len(resources)}")  # Debug log: Number of resources to visualize
for resource in resources:
    visualize_resource_plotly(resource, html_file, nodes_csv, edges_csv)

# Close the HTML file after writing all content
with open(html_file, 'a') as f:
    f.write("</body></html>")
    print(f"Closed HTML file: {html_file}")  # Debug log: HTML file closed

print(f"Graphs saved in {html_file}, nodes in {nodes_csv}, and edges in {edges_csv}")

Initialized HTML file: merged_knowledge_graphs.html
Fetching data from URL: https://api.datacite.org/dois?resource-type-id=study-registration&page[size]=20
API response status code: 200
Parsed 20 resources.
Total number of resources available: 125345
Number of resources fetched: 20
Number of resources to visualize: 20
Added central node: StudyRegistration:<br>DOI: 10.17605/osf.io/rmtfk <br> Resource Type: Pre-registration <br> SchemaOrg: CreativeWork
Added creator node and edge: Creator: Hsu, Emily <br> ID: https://orcid.org/0009-0003-6319-5909
Added contributor node and edge: Contributor: Center For Open Science <br> Type: HostingInstitution <br> ID: https://cos.io/
Added publisher node and edge: Publisher: <br> OSF Registries
Added related item node and edge: IsVersionOf: <br> https://osf.io/jgtn3


Displayed knowledge graph for resource: 10.17605/osf.io/rmtfk
Appended graph to HTML file: merged_knowledge_graphs.html
Appended nodes to CSV file: merged_nodes.csv
Appended edges to CSV file: merged_edges.csv
Added central node: StudyRegistration:<br>DOI: 10.17605/osf.io/bc4t8 <br> Resource Type: Pre-registration <br> SchemaOrg: CreativeWork
Added creator node and edge: Creator: Dawson, Ian G. J. <br> ID: https://orcid.org/0000-0003-0555-9682
Added contributor node and edge: Contributor: Center For Open Science <br> Type: HostingInstitution <br> ID: https://cos.io/
Added publisher node and edge: Publisher: <br> OSF Registries
Added related item node and edge: IsVersionOf: <br> https://osf.io/b2gha


Displayed knowledge graph for resource: 10.17605/osf.io/bc4t8
Appended graph to HTML file: merged_knowledge_graphs.html
Appended nodes to CSV file: merged_nodes.csv
Appended edges to CSV file: merged_edges.csv
Added central node: StudyRegistration:<br>DOI: 10.17605/osf.io/5x4sc <br> Resource Type: Pre-registration <br> SchemaOrg: CreativeWork
Added creator node and edge: Creator: Niranjan Ram <br> ID: https://osf.io/ev9xs
Added contributor node and edge: Contributor: Center For Open Science <br> Type: HostingInstitution <br> ID: https://cos.io/
Added publisher node and edge: Publisher: <br> OSF Registries
Added related item node and edge: IsVersionOf: <br> https://osf.io/8hx5d


Displayed knowledge graph for resource: 10.17605/osf.io/5x4sc
Appended graph to HTML file: merged_knowledge_graphs.html
Appended nodes to CSV file: merged_nodes.csv
Appended edges to CSV file: merged_edges.csv
Added central node: StudyRegistration:<br>DOI: 10.17605/osf.io/85q4u <br> Resource Type: Pre-registration <br> SchemaOrg: CreativeWork
Added creator node and edge: Creator: Stenzel, Kilian <br> ID: https://osf.io/mf97r
Added creator node and edge: Creator: Weiß, Martin <br> ID: http://scholar.google.com/citations?user=hxu937QAAAAJ
Added contributor node and edge: Contributor: Center For Open Science <br> Type: HostingInstitution <br> ID: https://cos.io/
Added publisher node and edge: Publisher: <br> OSF Registries
Added related item node and edge: IsIdenticalTo: <br> https://archive.org/details/osf-registrations-85q4u-v1
Added related item node and edge: IsVersionOf: <br> https://osf.io/e6rkc


Displayed knowledge graph for resource: 10.17605/osf.io/85q4u
Appended graph to HTML file: merged_knowledge_graphs.html
Appended nodes to CSV file: merged_nodes.csv
Appended edges to CSV file: merged_edges.csv
Added central node: StudyRegistration:<br>DOI: 10.17605/osf.io/vp6ty <br> Resource Type: Pre-registration <br> SchemaOrg: CreativeWork
Added creator node and edge: Creator: Tamminga, Sietske J. <br> ID: https://osf.io/qpfrg
Added creator node and edge: Creator: Zwanenburg, Bram <br> ID: https://osf.io/8tbq9
Added creator node and edge: Creator: Lindhout, Eva <br> ID: https://osf.io/qxsdc
Added creator node and edge: Creator: Jansen, Tom <br> ID: https://osf.io/dphe2
Added creator node and edge: Creator: Van Der Molen, Henk F. <br> ID: https://orcid.org/0000-0002-0719-2020
Added creator node and edge: Creator: Timmermans, Danielle <br> ID: http://researcherid.com/rid/https://www.researchgate.net/profile/Danielle-Timmermans
Added contributor node and edge: Contributor: Center For O

...

Displayed knowledge graph for resource: 10.17605/osf.io/xf3eg
Appended graph to HTML file: merged_knowledge_graphs.html
Appended nodes to CSV file: merged_nodes.csv
Appended edges to CSV file: merged_edges.csv
Added central node: StudyRegistration:<br>DOI: 10.17605/osf.io/hqsab <br> Resource Type: Pre-registration <br> SchemaOrg: CreativeWork
Added creator node and edge: Creator: Westmark, April <br> ID: https://osf.io/m6jbc
Added contributor node and edge: Contributor: Center For Open Science <br> Type: HostingInstitution <br> ID: https://cos.io/
Added publisher node and edge: Publisher: <br> OSF Registries
Added related item node and edge: IsVersionOf: <br> https://osf.io/xv8nm


Displayed knowledge graph for resource: 10.17605/osf.io/hqsab
Appended graph to HTML file: merged_knowledge_graphs.html
Appended nodes to CSV file: merged_nodes.csv
Appended edges to CSV file: merged_edges.csv
Added central node: StudyRegistration:<br>DOI: 10.17605/osf.io/7zjwc <br> Resource Type: Pre-registration <br> SchemaOrg: CreativeWork
Added creator node and edge: Creator: Matlock, Michele L <br> ID: https://osf.io/5vuax
Added contributor node and edge: Contributor: Center For Open Science <br> Type: HostingInstitution <br> ID: https://cos.io/
Added publisher node and edge: Publisher: <br> OSF Registries
Added related item node and edge: IsVersionOf: <br> https://osf.io/jaef7


Displayed knowledge graph for resource: 10.17605/osf.io/7zjwc
Appended graph to HTML file: merged_knowledge_graphs.html
Appended nodes to CSV file: merged_nodes.csv
Appended edges to CSV file: merged_edges.csv
Added central node: StudyRegistration:<br>DOI: 10.17605/osf.io/cszuw <br> Resource Type: Pre-registration <br> SchemaOrg: CreativeWork
Added creator node and edge: Creator: Hales, Caz <br> ID: https://orcid.org/0000-0002-3197-1657
Added creator node and edge: Creator: Robinson, Brian <br> ID: https://osf.io/q57xd
Added creator node and edge: Creator: Dewar, Janine <br> ID: https://osf.io/jvzp8
Added contributor node and edge: Contributor: Center For Open Science <br> Type: HostingInstitution <br> ID: https://cos.io/
Added publisher node and edge: Publisher: <br> OSF Registries
Added related item node and edge: IsVersionOf: <br> https://osf.io/2qj3k


Displayed knowledge graph for resource: 10.17605/osf.io/cszuw
Appended graph to HTML file: merged_knowledge_graphs.html
Appended nodes to CSV file: merged_nodes.csv
Appended edges to CSV file: merged_edges.csv
Added central node: StudyRegistration:<br>DOI: 10.17605/osf.io/26w7x <br> Resource Type: Pre-registration <br> SchemaOrg: CreativeWork
Added creator node and edge: Creator: Song, Yibo <br> ID: https://orcid.org/0009-0008-7992-4519
Added contributor node and edge: Contributor: Center For Open Science <br> Type: HostingInstitution <br> ID: https://cos.io/
Added publisher node and edge: Publisher: <br> OSF Registries
Added related item node and edge: IsVersionOf: <br> https://osf.io/ej4ca


Displayed knowledge graph for resource: 10.17605/osf.io/26w7x
Appended graph to HTML file: merged_knowledge_graphs.html
Appended nodes to CSV file: merged_nodes.csv
Appended edges to CSV file: merged_edges.csv
Closed HTML file: merged_knowledge_graphs.html
Graphs saved in merged_knowledge_graphs.html, nodes in merged_nodes.csv, and edges in merged_edges.csv
